In [49]:
import enchant
from enchant.checker import SpellChecker
import pymongo as pym
import string
from nltk.corpus import stopwords
import stop_words
import pandas as pd
import numpy as np
from collections import Counter
import time
import gc
import re

In [50]:
stops = set(['rt','ds','qd','ss','ns','vs','nn','amp','gt','gd','gds','tt','pr','ac','mm', 'qu',
            '``', 'ni', 'ca', 'le', 'les', ' ', 'si', '$', '^', 'via', 'ils','pour','une','que','quel']
        + list('@ن%£€‘:&;')+ list('abcdefghijklmnopqrstuvwxyzà'))


In [51]:
def mongo_to_df(collection, n_last_tweets=0, retweet=False):
    tweets = collection.find(filter={'text':{'$exists':True}}, 
                             projection={'_id':False}).sort('$natural',-1).limit(n_last_tweets)
    df = pd.DataFrame()
    listTweets, listCandidats, listSentiments = [], [], []
    
    for t in tweets: 
        if not retweet: # filtrage des retweets
            if 'rt @' in t['text']:
                continue

        if t['text']: # test si liste non vide
            listTweets.append(t['text'])
            try:
                listCandidats.append(t['candidat'])
            except:
                listCandidats.append(None)
            
            try:
                listSentiments.append(t['sentiment'])
            except:
                listSentiments.append(None)
    
    df['text'], df['candidat'], df['sentiment'] = listTweets, listCandidats, listSentiments
    return df

In [53]:
client = pym.MongoClient('localhost',27018)
collection = client.tweet.train
print('{} tweets in the manual train set.'.format(collection.count()))
df_tweets = mongo_to_df(collection, n_last_tweets=0, retweet=True)
print(df_tweets['sentiment'].value_counts())
print(df_tweets['candidat'].value_counts())
df_tweets.head(5)
collection = client.tweet.labelised
print('{} tweets in the auto train set.'.format(collection.count()))
df_tweets_auto = mongo_to_df(collection, n_last_tweets=0, retweet=False)
print(df_tweets_auto['sentiment'].value_counts())
print(df_tweets_auto['candidat'].value_counts())
df_tweets_auto.head(5)

10002 tweets in the manual train set.
-1.0    5339
 0.0    3456
 1.0    1206
Name: sentiment, dtype: int64
Series([], Name: candidat, dtype: int64)
1421 tweets in the auto train set.
1    1418
Name: sentiment, dtype: int64
fillon       299
macron       289
le pen       288
hamon        274
melenchon    268
Name: candidat, dtype: int64


text   candidat  sentiment
0  même s'il veut faire croire qu'il l'adoucit le...  melenchon          1
1  si #fillon pas présent second tour #mlp sera m...  melenchon          1
2  #fillon et #macron sont au coude à coude dans ...  melenchon          1
3  @edfofficiel votre pub est à chier, où avait v...  melenchon          1
4  @sofiakkar faut pas pousser  #fillon  parle de...  melenchon          1

In [54]:
def regex_filter(text):
    text = re.sub(r'\w*…', '', text) # mot tronqué par Twitter
    text = re.sub(r'(?:htt)\S*', '', text) # retrait des liens http
    text = re.sub(r'\n', ' ', text) # retrait des sauts de ligne
    text = re.sub(r'rt\s','',text)
    text = re.sub(r'gt\s','',text)
    text = re.sub(r'\xad', '-', text)
    text = re.sub(r'\.{3,}', '...', text) # ....... => points de suspension
    text = re.sub(r'(?=\.\w)(\.)', '. ', text) # remplacer un point entre deux mots 'A.B' par 'A. B'
    return text

def process_texts(list_of_texts):
    dic = enchant.Dict('fr_FR') # Dictionnaire pour le spellchecking
    ignore_words = ['fillon','macron','mélenchon','melenchon','asselineau','poutou','arthaud','dupont-aignan',
                    'hamon','valls','hollande','mlp','le pen','pen', 'lepen','france','bfm','sarkozy','bayrou','servier',
                    'estrosi','retailleau','emmanuel','françois','marine','benoît','jean-luc','jadot','juppé','rtl','morano',
                    'collomb','lepage','hue','borloo','fillonistes','dutreil','UE','ue','l\'ue','mandon','essonne',
                    'fenech','baroin','verhofstadt','drahi','draghi','jlm','pécresse','pecresse','merkel','erdogan','trump',
                    'clinton','may','sturgeon','montebourg','lassalle','cheminade','rugy','trocadero','trocadéro','pinel',
                    'taubira','lafarge','axa','poutine','rothschild','cahuzac','bourgi','elkhomri','cgt','medef','cgpme',
                    'ofce','ocde','fmi','bce','smic','cice','cevipof','soprasteria','ifop','présitrack','ciotti','solère',
                   'solere','cambadélis','cambadelis','debré','delanoé','stefanini','angela','itélé','cir','pnf','dlf','lr',
                   'fn','amp','alain','bruno','mdr','lol','mme','sarko','gaulle','l\'udi','tweet','tweets','juppe','lejdd',
                    'penelopegate','penelope','mr','er','tf','isf','ff','tf','retweeted','bfmacron','wauquiez','jl','ceta',
                    'ttip','tafta','nda','jdd','guaino','élysée','l\'élysée','philippot','jt','copé','cohn','bergé','sarthe',
                    'svp','ls','replay','bfmtv','qq','hollandisme','bcp','balkany','peillon','attali','khomri','cazeneuve',
                    'lagarde','bendit','truchot','franceinfo',
                    'cqfd','rmc','france2','tf1','figaro','mediapart','médiapart','onpc','kkkkkkkkkkkkkkkkul','eelv','psg']
    list_of_processed_texts = []
    misspellings = []
    i = 0
    for text in list_of_texts:
        startTime = time.time()
        text = regex_filter(text)
        chkr = SpellChecker('fr_FR', text)
        for w in ignore_words : chkr.ignore_always(w)
        for err in chkr :
            if err.leading_context(1) == "#" or err.leading_context(1) == "@": 
                continue
            misspellings.append(err.word)
            try : err.replace(chkr.suggest(err.word)[0])
            except : continue
        text = chkr.get_text()
        list_of_processed_texts.append(text)
        i += 1
        if i%500 == 0 : print(str(i) + ' tweets traités en ', (time.time() - startTime)*60, ' secondes')
    print(Counter(misspellings).most_common(100))
    gc.collect()
    return list_of_processed_texts

In [55]:
tweet_list = process_texts(df_tweets['text'])
try:
    tweet_list.extend(process_texts(df_tweets_auto['text']))
except:
    pass

500 tweets traités en  0.08637428283691406  secondes
1000 tweets traités en  40.30832290649414  secondes
1500 tweets traités en  0.11478424072265625  secondes
2000 tweets traités en  3.148512840270996  secondes
2500 tweets traités en  0.14731407165527344  secondes
3000 tweets traités en  60.006065368652344  secondes
3500 tweets traités en  18.355393409729004  secondes
4000 tweets traités en  0.06115436553955078  secondes
4500 tweets traités en  7.604470252990723  secondes
5000 tweets traités en  33.92390727996826  secondes
5500 tweets traités en  23.51264476776123  secondes
6000 tweets traités en  0.09057998657226562  secondes
6500 tweets traités en  17.907814979553223  secondes
7000 tweets traités en  0.09754657745361328  secondes
7500 tweets traités en  0.9828329086303711  secondes
8000 tweets traités en  0.03311634063720703  secondes
8500 tweets traités en  0.062270164489746094  secondes
9000 tweets traités en  0.08409976959228516  secondes
9500 tweets traités en  0.1090764999389648

In [56]:
print(df_tweets.head(5))
print(df_tweets_auto.head(5))
df_all = df_tweets
#df_all.reset_index(inplace=True)
df_all = df_all.append(df_tweets_auto, ignore_index=True)
print(df_all.tail(5))
print(len(tweet_list))
df_all['text'] = tweet_list
print(df_all.tail(5))

                                                text candidat  sentiment
0  #le seul projet solide et construit pour nous ...     None        1.0
1  chirac mort, fillon assassiné. la droite fantôme.     None       -1.0
2  donc le politique a confiance en la justice qu...     None       -1.0
3  ce pays me dégoûte ! ces vieilles fientes qui ...     None       -1.0
4  laboratoire servier ?....le conseiller santé d...     None       -1.0
                                                text   candidat  sentiment
0  même s'il veut faire croire qu'il l'adoucit le...  melenchon          1
1  si #fillon pas présent second tour #mlp sera m...  melenchon          1
2  #fillon et #macron sont au coude à coude dans ...  melenchon          1
3  @edfofficiel votre pub est à chier, où avait v...  melenchon          1
4  @sofiakkar faut pas pousser  #fillon  parle de...  melenchon          1
                                                    text candidat  sentiment
11414  macron : girouette ou anguil

In [48]:
print(df_all['text'].ix[-5:], df_all['sentiment'].ix[-5:])

0        #le seul projet solide et construit pour nous ...
1        Chirac mort, fillon assassiné. la droite fantôme.
2        donc le politique a confiance en la justice qu...
3        ce pays me dégoûte ! ces vieilles fientes qui ...
4        laboratoire servier ?... le conseiller santé d...
5        #bayrou avec #macron, en marche les politicien...
6        👉@lesrepublicains  de 👉fillon à estrosi en pas...
7        des économistes jugent l’équilibre du projet d...
8        @marineelysee et macron accepte cette merde fa...
9        les fillonistes ont été tellement amaigris par...
10       ces socialistes qui apportent leur soutien à e...
11       précipitée #fillon nous vole depuis 40 ans, et...
12       #macron demande d'aller vers la république des...
13       @molinarijulie @dominiquereynie @ifopopinion @...
14       déjeuner avec sarko lui a fait un bien! #ilnou...
15       @dlf_officiel @dupontaignan @tf1 les médias fr...
16            conférence de presse de fillon : allégorie

In [60]:
collection = client.tweet.spellchecked
collection.insert_many(df_all.to_dict('records'))